In [12]:
import numpy as np
import pandas as pd

In [13]:
features = pd.read_csv('0325_vars_final_zscale.csv')

In [14]:
oot = pd.read_csv('0325_oot_final_zscale.csv')

In [15]:
features.head()

,Recnum,Fraud,Actual/mean_Cardnum_30d,Actual/mean_Merchnum_14d,Actual/mean_Merchnum_30d,Actual/mean_Merchnum_7d,Actual/median_Merchnum_7d,Actual/sum_Cardnum_Merch zip_3d,Actual/sum_Cardnum_Merchnum_3d,Days_since_per_Cardnum,...,count_Cardnum_7d,count_Cardnum_Merch state_3d,count_Cardnum_Merch state_7d,count_Cardnum_Merch zip_3d,count_Cardnum_Merchnum_3d,count_Cardnum_Merchnum_7d,mean_Cardnum_1d,Actual/count_Cardnum_1d,Actual/count_Cardnum_30d,mean_Cardnum_30d
0,818,0,-0.003287,0.001750,0.001823,-0.000786,-0.031299,0.529859,0.505899,7.346025,...,-0.368917,-0.227509,-0.212899,-0.207194,-0.204367,-0.185907,-0.474089,-0.414382,-0.077371,-0.721970
1,1403,0,0.915446,0.338513,0.263935,0.374613,-0.013802,0.529859,0.505899,-0.202011,...,-0.178355,-0.227509,-0.212899,-0.207194,-0.204367,-0.185907,0.091413,0.229728,0.219330,-0.437000
2,1222,0,-0.081736,-0.232596,-0.180575,-0.262019,-0.031299,0.529859,0.505899,-0.202011,...,-0.305396,-0.107245,-0.148293,-0.207194,-0.204367,-0.185907,-0.421739,-0.354755,-0.101906,-0.617050
3,2169,0,-0.125585,-0.155362,-0.120462,-0.175923,-0.028772,0.529859,0.505899,-0.139801,...,-0.178355,-0.227509,-0.212899,-0.207194,-0.204367,-0.185907,-0.292151,-0.404932,-0.173845,-0.539719
4,2090,0,-0.411285,-0.343948,-0.267243,-0.386146,-0.046741,0.529859,0.505899,-0.139801,...,-0.305396,-0.227509,-0.212899,-0.207194,-0.204367,-0.185907,-0.507067,-0.451944,-0.241247,-0.674160


In [16]:
features.shape

(83970, 22)

In [17]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(features['Fraud'])
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('Fraud', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [18]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [19]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (62977, 21)
Training Labels Shape: (62977,)
Testing Features Shape: (20993, 21)
Testing Labels Shape: (20993,)


In [20]:
#### Import the model we are using
from sklearn.ensemble import RandomForestRegressor

FDR = []

estimators = [20,400,1000]
maxdepth = [10,15]
minsamplesleaf = [5,10]

for e in estimators:
    for md in maxdepth:
        for msl in minsamplesleaf:
            # Instantiate model with 400 decision trees
            rf = RandomForestRegressor(n_estimators = e, max_depth = md, min_samples_leaf = msl, random_state = 42)
            # Train the model on training data
            rf.fit(train_features, train_labels)
            # Use the forest's predict method on the test data
            predictions = rf.predict(test_features)


            ## FDR train
            train_data = pd.concat([pd.DataFrame(train_features),pd.Series(train_labels),pd.Series(rf.predict(train_features))], axis = 1)
            train_data.columns = ['Recnum', 'Actual/mean_Cardnum_30d',
                   'Actual/mean_Merchnum_14d', 'Actual/mean_Merchnum_30d',
                   'Actual/mean_Merchnum_7d', 'Actual/median_Merchnum_7d',
                   'Actual/sum_Cardnum_Merch zip_3d', 'Actual/sum_Cardnum_Merchnum_3d',
                   'Days_since_per_Cardnum', 'Days_since_per_Cardnum_Merch state',
                   'count_Cardnum_1d', 'count_Cardnum_7d', 'count_Cardnum_Merch state_3d',
                   'count_Cardnum_Merch state_7d', 'count_Cardnum_Merch zip_3d',
                   'count_Cardnum_Merchnum_3d', 'count_Cardnum_Merchnum_7d',
                   'mean_Cardnum_1d', 'Actual/count_Cardnum_1d',
                   'Actual/count_Cardnum_30d', 'mean_Cardnum_30d','Fraud','pred']

            topRows = int(round(len(train_data)*0.03)) # 3% of population

            temp = train_data.sort_values('pred', ascending = False).head(topRows)
            needed = temp.loc[:,'Fraud']
            FDR_train = sum(needed)/sum(train_data['Fraud'])


            ## FDR test
            test_data = pd.concat([pd.DataFrame(test_features),pd.Series(test_labels),pd.Series(predictions)], axis = 1)
            test_data.columns = ['Recnum', 'Actual/mean_Cardnum_30d',
                   'Actual/mean_Merchnum_14d', 'Actual/mean_Merchnum_30d',
                   'Actual/mean_Merchnum_7d', 'Actual/median_Merchnum_7d',
                   'Actual/sum_Cardnum_Merch zip_3d', 'Actual/sum_Cardnum_Merchnum_3d',
                   'Days_since_per_Cardnum', 'Days_since_per_Cardnum_Merch state',
                   'count_Cardnum_1d', 'count_Cardnum_7d', 'count_Cardnum_Merch state_3d',
                   'count_Cardnum_Merch state_7d', 'count_Cardnum_Merch zip_3d',
                   'count_Cardnum_Merchnum_3d', 'count_Cardnum_Merchnum_7d',
                   'mean_Cardnum_1d', 'Actual/count_Cardnum_1d',
                   'Actual/count_Cardnum_30d', 'mean_Cardnum_30d','Fraud','pred']

            topRows = int(round(len(test_data)*0.03)) # 3% of population

            temp = test_data.sort_values('pred', ascending = False).head(topRows)
            needed = temp.loc[:,'Fraud']
            FDR_test = sum(needed)/sum(test_data['Fraud'])

            ## FDR oot
            oot_data = pd.concat([pd.DataFrame(oot),pd.Series(rf.predict(oot.drop('Fraud',axis = 1)))], axis = 1)
            oot_data.columns = ['Recnum','Fraud', 'Actual/mean_Cardnum_30d',
                   'Actual/mean_Merchnum_14d', 'Actual/mean_Merchnum_30d',
                   'Actual/mean_Merchnum_7d', 'Actual/median_Merchnum_7d',
                   'Actual/sum_Cardnum_Merch zip_3d', 'Actual/sum_Cardnum_Merchnum_3d',
                   'Days_since_per_Cardnum', 'Days_since_per_Cardnum_Merch state',
                   'count_Cardnum_1d', 'count_Cardnum_7d', 'count_Cardnum_Merch state_3d',
                   'count_Cardnum_Merch state_7d', 'count_Cardnum_Merch zip_3d',
                   'count_Cardnum_Merchnum_3d', 'count_Cardnum_Merchnum_7d',
                   'mean_Cardnum_1d', 'Actual/count_Cardnum_1d',
                   'Actual/count_Cardnum_30d', 'mean_Cardnum_30d','pred']

            topRows = int(round(len(oot_data)*0.03)) # 3% of population

            temp = oot_data.sort_values('pred', ascending = False).head(topRows)
            needed = temp.loc[:,'Fraud']
            FDR_oot = sum(needed)/sum(oot_data['Fraud'])

            FDR.append([FDR_train,FDR_test,FDR_oot])

In [21]:
FDR = pd.DataFrame(FDR, columns = ['train','test','oot'])

In [22]:
FDR

,train,test,oot
0,0.904690,0.831050,0.374302
1,0.889561,0.821918,0.357542
2,0.989410,0.853881,0.391061
3,0.978820,0.844749,0.351955
4,0.915280,0.858447,0.374302
5,0.892587,0.849315,0.374302
6,0.995461,0.853881,0.374302
7,0.981846,0.849315,0.368715
8,0.915280,0.849315,0.379888
9,0.897126,0.840183,0.374302


In [42]:
FDR.drop([0,1],axis = 0, inplace = True)
FDR.loc['mean',:] = FDR.mean(axis = 0)
FDR

,train,test,oot
2,0.989410,0.853881,0.391061
3,0.978820,0.844749,0.351955
4,0.915280,0.858447,0.374302
5,0.892587,0.849315,0.374302
6,0.995461,0.853881,0.374302
7,0.981846,0.849315,0.368715
8,0.915280,0.849315,0.379888
9,0.897126,0.840183,0.374302
10,0.995461,0.849315,0.379888
11,0.981846,0.853881,0.374302


In [26]:
rf = RandomForestRegressor(n_estimators = 20, max_depth = 15, min_samples_leaf = 5, random_state = 42)
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [44]:
# train-FDR (1-50)
train_data = pd.concat([pd.DataFrame(train_features),pd.Series(train_labels),pd.Series(rf.predict(train_features))], axis = 1)
train_data.columns = ['Recnum', 'Actual/mean_Cardnum_30d',
                   'Actual/mean_Merchnum_14d', 'Actual/mean_Merchnum_30d',
                   'Actual/mean_Merchnum_7d', 'Actual/median_Merchnum_7d',
                   'Actual/sum_Cardnum_Merch zip_3d', 'Actual/sum_Cardnum_Merchnum_3d',
                   'Days_since_per_Cardnum', 'Days_since_per_Cardnum_Merch state',
                   'count_Cardnum_1d', 'count_Cardnum_7d', 'count_Cardnum_Merch state_3d',
                   'count_Cardnum_Merch state_7d', 'count_Cardnum_Merch zip_3d',
                   'count_Cardnum_Merchnum_3d', 'count_Cardnum_Merchnum_7d',
                   'mean_Cardnum_1d', 'Actual/count_Cardnum_1d',
                   'Actual/count_Cardnum_30d', 'mean_Cardnum_30d','Fraud','pred']
train_data.sort_values('pred', ascending = False, inplace = True)
GoodTest=sum(train_data['Fraud']==0)
BadTest=sum(train_data['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0

for i in range(100):
    data=train_data.iloc[r:min(int((i+1)*len(train_data)/100),len(train_data)),]
    populationbin=i+ 1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min(int((i+1)*len(train_data)/100),len(train_data))-r
    totalrecord=min(int((i+1)*len(train_data)/100),len(train_data))
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r= min(int((i+1)*len(train_data)/100),len(train_data))
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
l.to_excel('train-FDR.xlsx')
l.head(50)

,populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR
0,1,629,81,548,0.129983,82.904690,629,81,548,0.129983,82.904690,82.774707,0.001568
1,2,630,543,87,0.871365,13.161876,1259,624,635,1.001348,96.066566,95.065218,0.010423
2,3,630,611,19,0.980487,2.874433,1889,1235,654,1.981835,98.940998,96.959164,0.020030
3,4,630,625,5,1.002953,0.756430,2519,1860,659,2.984787,99.697428,96.712641,0.029938
4,5,629,629,0,1.009372,0.000000,3148,2489,659,3.994159,99.697428,95.703269,0.040063
5,6,630,628,2,1.007767,0.302572,3778,3117,661,5.001926,100.000000,94.998074,0.050019
6,7,630,630,0,1.010976,0.000000,4408,3747,661,6.012902,100.000000,93.987098,0.060129
7,8,630,630,0,1.010976,0.000000,5038,4377,661,7.023878,100.000000,92.976122,0.070239
8,9,629,629,0,1.009372,0.000000,5667,5006,661,8.033250,100.000000,91.966750,0.080332
9,10,630,630,0,1.010976,0.000000,6297,5636,661,9.044226,100.000000,90.955774,0.090442


In [45]:
# test-FDR (1-50)
test_data = pd.concat([pd.DataFrame(test_features),pd.Series(test_labels),pd.Series(predictions)], axis = 1)
test_data.columns = ['Recnum', 'Actual/mean_Cardnum_30d',
       'Actual/mean_Merchnum_14d', 'Actual/mean_Merchnum_30d',
       'Actual/mean_Merchnum_7d', 'Actual/median_Merchnum_7d',
       'Actual/sum_Cardnum_Merch zip_3d', 'Actual/sum_Cardnum_Merchnum_3d',
       'Days_since_per_Cardnum', 'Days_since_per_Cardnum_Merch state',
       'count_Cardnum_1d', 'count_Cardnum_7d', 'count_Cardnum_Merch state_3d',
       'count_Cardnum_Merch state_7d', 'count_Cardnum_Merch zip_3d',
       'count_Cardnum_Merchnum_3d', 'count_Cardnum_Merchnum_7d',
       'mean_Cardnum_1d', 'Actual/count_Cardnum_1d',
       'Actual/count_Cardnum_30d', 'mean_Cardnum_30d','Fraud','pred']

test_data.sort_values('pred', ascending = False, inplace = True)
GoodTest=sum(test_data['Fraud']==0)
BadTest=sum(test_data['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0


for i in range(100):
    data=test_data.iloc[r:min(int((i+1)*len(test_data)/100),len(test_data)),]
    populationbin=i+ 1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min(int((i+1)*len(test_data)/100),len(test_data))-r
    totalrecord=min(int((i+1)*len(test_data)/100),len(test_data))
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r= min(int((i+1)*len(test_data)/100),len(test_data))
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
l.to_excel('test-FDR.xlsx')
l.head(50)

,populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR
0,1,209,63,146,0.303264,66.666667,209,63,146,0.303264,66.666667,66.363403,0.004549
1,2,210,175,35,0.842399,15.981735,419,238,181,1.145663,82.648402,81.502739,0.013862
2,3,210,204,6,0.981997,2.739726,629,442,187,2.127660,85.388128,83.260468,0.024918
3,4,210,209,1,1.006065,0.456621,839,651,188,3.133725,85.844749,82.711024,0.036505
4,5,210,206,4,0.991624,1.826484,1049,857,192,4.125349,87.671233,83.545884,0.047055
5,6,210,207,3,0.996438,1.369863,1259,1064,195,5.121787,89.041096,83.919309,0.057522
6,7,210,208,2,1.001252,0.913242,1469,1272,197,6.123038,89.954338,83.831299,0.068068
7,8,210,210,0,1.010879,0.000000,1679,1482,197,7.133917,89.954338,82.820421,0.079306
8,9,210,209,1,1.006065,0.456621,1889,1691,198,8.139983,90.410959,82.270976,0.090033
9,10,210,207,3,0.996438,1.369863,2099,1898,201,9.136421,91.780822,82.644401,0.099546


In [46]:
# oot-FDR (1-50)
oot_data = pd.concat([pd.DataFrame(oot),pd.Series(rf.predict(oot.drop('Fraud',axis = 1)))], axis = 1)
oot_data.columns = ['Recnum','Fraud', 'Actual/mean_Cardnum_30d',
       'Actual/mean_Merchnum_14d', 'Actual/mean_Merchnum_30d',
       'Actual/mean_Merchnum_7d', 'Actual/median_Merchnum_7d',
       'Actual/sum_Cardnum_Merch zip_3d', 'Actual/sum_Cardnum_Merchnum_3d',
       'Days_since_per_Cardnum', 'Days_since_per_Cardnum_Merch state',
       'count_Cardnum_1d', 'count_Cardnum_7d', 'count_Cardnum_Merch state_3d',
       'count_Cardnum_Merch state_7d', 'count_Cardnum_Merch zip_3d',
       'count_Cardnum_Merchnum_3d', 'count_Cardnum_Merchnum_7d',
       'mean_Cardnum_1d', 'Actual/count_Cardnum_1d',
       'Actual/count_Cardnum_30d', 'mean_Cardnum_30d','pred']

oot_data.sort_values('pred', ascending = False, inplace = True)
GoodTest=sum(oot_data['Fraud']==0)
BadTest=sum(oot_data['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0


for i in range(100):
    data=oot_data.iloc[r:min(int((i+1)*len(oot_data)/100),len(oot_data)),]
    populationbin=i+ 1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min(int((i+1)*len(oot_data)/100),len(oot_data))-r
    totalrecord=min(int((i+1)*len(oot_data)/100),len(oot_data))
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r= min(int((i+1)*len(oot_data)/100),len(oot_data))
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
l.to_excel('oot-FDR.xlsx')
l.head(50)

,populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR
0,1,124,79,45,0.645003,25.139665,124,79,45,0.645003,25.139665,24.494662,0.025657
1,2,124,112,12,0.914435,6.703911,248,191,57,1.559438,31.843575,30.284137,0.048972
2,3,124,111,13,0.906270,7.262570,372,302,70,2.465709,39.106145,36.640437,0.063052
3,4,125,118,7,0.963423,3.910615,497,420,77,3.429131,43.016760,39.587628,0.079716
4,5,124,120,4,0.979752,2.234637,621,540,81,4.408883,45.251397,40.842514,0.097431
5,6,124,123,1,1.004246,0.558659,745,663,82,5.413129,45.810056,40.396927,0.118165
6,7,124,120,4,0.979752,2.234637,869,783,86,6.392880,48.044693,41.651812,0.133061
7,8,125,118,7,0.963423,3.910615,994,901,93,7.356303,51.955307,44.599004,0.141589
8,9,124,119,5,0.971587,2.793296,1118,1020,98,8.327890,54.748603,46.420713,0.152111
9,10,124,119,5,0.971587,2.793296,1242,1139,103,9.299477,57.541899,48.242422,0.161612


In [56]:
data = [train_data, test_data, oot_data]
summary = []
for set in data:
    summary.append([len(set),(len(set)-sum(set['Fraud'])),sum(set['Fraud']),sum(set['Fraud'])/len(set)])

summary = pd.DataFrame(summary, columns = ['#Records','#Goods','#Bads','Fraud Rate'], index = ['train','test','oot'])
summary.to_excel('data_summary.xlsx')
summary

,#Records,#Goods,#Bads,Fraud Rate
train,62977,62316,661,0.010496
test,20993,20774,219,0.010432
oot,12427,12248,179,0.014404
